# Predict the stock market with news

In [8]:
# Set up the environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pandas as pd
import numpy as np
import gensim.models
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing import Tokenizer, text_to_word_sequence                                                                                                      from sklearn.preprocessing import FunctionTransformer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

# NLP libraries
from textblob import TextBlob

In [45]:
import sys
sys.path.append('../')
from STOCK_PREDICT.get_data import get_data
df_djia, df_news = get_data(1989)

In [58]:
df_news.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."


## Data cleaning

In [61]:
# combine the top 25 news
cols = df_news.columns[2:27]
df_news['combined'] = df_news[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [63]:
df_news.shape

(1989, 28)

In [65]:
df_news_comb = df_news

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,combined
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge""","b""Georgia 'downs two Russian warplanes' as cou..."


In [59]:
import sys
from STOCK_PREDICT.df import clean
df_news['cleaned'] = df_news['combined'].apply(clean)

KeyError: 'combined'

In [55]:
sys.path.append('../')

## Feature creation with Textblob

References : 
https://textblob.readthedocs.io/en/dev/quickstart.html#create-a-textblob


#### Textblobbing

In [9]:
#Textblob
wiki = TextBlob(df['combined'][0])

# Subjectivity and polarity
polarity = wiki.sentiment[0]
subjectivity = wiki.sentiment[1]
polarity, subjectivity

NameError: name 'df' is not defined

#### Textblob function

In [ ]:
def polarity(text):
    blob = TextBlob(text)
    polarity = blob.sentiment[0]
    return polarity

def subjectivity(text):
    blob = TextBlob(text)
    subjectivity = blob.sentiment[1]
    return subjectivity

In [ ]:
# Feature creation
df['polarity'] = df['combined'].apply(polarity)
df['subjectivity'] = df['combined'].apply(subjectivity)

# Merge data

# Add target column and % change column

In [16]:
data.drop('index', axis=1, inplace=True)

In [17]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top23,Top24,Top25,Open,High,Low,Close,Volume,Adj Close,change
0,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961,-0.012637
1,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,"b'Russia: World ""can forget about"" Georgia\'s...",b'Darfur rebels accuse Sudan of mounting major...,b'Philippines : Peace Advocate say Muslims nee...,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688,-0.008660
2,2008-08-15,1,"b""Mom of missing gay man: Too bad he's not a 2...","b""Russia: U.S. Poland Missile Deal Won't Go 'U...","b""The government has been accused of creating ...",b'The Italian government has lashed out at an ...,b'Gorbachev: Georgia started conflict in S. Os...,"b""China fakes more than your girlfriend; 'Ethn...","b""The UN's criticism of freedom of expression ...",b'Russian general threatens nuclear strike on ...,...,b'Bush Tells Putin: This Aggression Will Not S...,b'Georgia is all about the oil pipelines',b'Rivals say they plan to remove Georgian pres...,11611.209961,11709.889648,11599.730469,11659.900391,215040000,11659.900391,0.006863
3,2008-08-18,0,"b'In an Afghan prison, the majority of female ...","b""Little girl, you're not ugly; they are""","b""Pakistan's Musharraf to Resign, Leave the Co...","b'Tornado throws a bus in Poland, captured by ...","b""Britain's terror laws have left me and my fa...","b""Iran 'fires satellite into space'""",b'Rights of Non-Muslims restricted by new Mald...,b'Tour of Tskhinvali undercuts Russian version...,...,b'Putin has given us an order that everyone mu...,b'National DNA database grows on the genes of ...,b'Mayor Asks Ugly Women To Visit His Town',11659.650391,11690.429688,11434.120117,11479.389648,156290000,11479.389648,0.004172
4,2008-08-19,0,"b""Man arrested and locked up for five hours af...",b'The US missile defence system is the magic p...,b'Schrder lambasted for blaming Russian confli...,b'Officials: 10 French soldiers killed near Ka...,b'These ten laws make China a totalitarian was...,b'Russia seizes US vehicles',"b""Muslims are only 4% of Denmark's 5.4 million...",b'Taliban Forces Kill 10 French Soldiers and R...,...,b'Taliban Forces Kill 10 French Soldiers and R...,b'Pakistan is more democratic than America. ',b'Blaze engulfs Egyptian parliament',11478.089844,11478.169922,11318.500000,11348.549805,171580000,11348.549805,-0.015572


In [18]:
# Get categorical data
def categorical(x):
    if x > 0:
        x = 1
    else:
        x = 0
    return x

In [19]:
data['target'] = data['change'].apply(categorical)



In [20]:
# Combine the top 25 daily news into 1 column
cols = data.columns[2:27]
data['combined'] = data[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# Naive_base avec model avec column combined

In [27]:
df = data[['Date', 'combined']]

In [28]:
df.head()

,Date,combined
0,2008-08-13,b' U.S. refuses Israel weapons to attack Iran:...
1,2008-08-14,b'All the experts admit that we should legalis...
2,2008-08-15,"b""Mom of missing gay man: Too bad he's not a 2..."
3,2008-08-18,"b'In an Afghan prison, the majority of female ..."
4,2008-08-19,"b""Man arrested and locked up for five hours af..."


In [23]:
def clean(text):
    for punctuation in string.punctuation:
        review1 = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
#     if re.search(r'^b\s', text):
#         text = text.replace("b ","")
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return " ".join(lemmatized)

df['cleaned'] = df['combined'].apply(clean)

NameError: name 'df' is not defined

In [28]:
import sys
sys.path.append('../')
from STOCK_PREDICT.df import get_data
df_djia, df_news = get_data()

ModuleNotFoundError: No module named 'df'

In [ ]:
USERNAME='Jean-Phi-ben'
def get_data(nrows):
    path1=f'/Users/{USERNAME}/code/{USERNAME}/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv'
    path2=f'/Users/{USERNAME}/code/{USERNAME}/STOCK_PREDICT/data/Combined_News_DJIA.csv'
    df_djia = pd.read_csv(path1, nrows=nrows)
    df_news = pd.read_csv(path2,nrows=nrows)
    return df_djia, df_news

In [13]:
pwd

'/home/jp/code/STOCK_PREDICT/Notebooks'

In [35]:
import sys
sys.path.append('../')
from STOCK_PREDICT.df import get_data
df_djia, df_news = get_data(1989)

FileNotFoundError: [Errno 2] File b'/Users/Jean-Phi-ben/code/Jean-Phi-ben/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv' does not exist: b'/Users/Jean-Phi-ben/code/Jean-Phi-ben/STOCK_PREDICT/data/datasets-129-792900-upload_DJIA_table.csv'

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jp/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True